In [1]:
import numpy as np
import sys,sklearn
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['font.family'] = 'AppleGothic'

In [2]:
df = pd.read_csv('NHIS_OPEN_GJ_2017_100.csv',engine = 'python')

In [3]:
df_s = df[['성별코드','신장(5Cm단위)', '체중(5Kg단위)', '연령대코드(5세단위)','이완기혈압','수축기혈압','식전혈당(공복혈당)', '허리둘레']]

In [4]:
df_s = df_s.rename(columns={"연령대코드(5세단위)":"연령대코드",
                            "체중(5Kg단위)":"체중",
                            "신장(5Cm단위)":"신장",             
                            "식전혈당(공복혈당)":"식전혈당"})

In [5]:
df_s.head(5)

,성별코드,신장,체중,연령대코드,이완기혈압,수축기혈압,식전혈당,허리둘레
0,1,170,75,8,80.0,120.0,99.0,90.0
1,1,180,80,7,82.0,130.0,106.0,89.0
2,1,165,75,9,70.0,120.0,98.0,91.0
3,1,175,80,11,87.0,145.0,95.0,91.0
4,1,165,60,11,82.0,138.0,101.0,80.0


## 허리둘레(40이상 140 이하) 

In [6]:
df_s = df_s.loc[(df_s['허리둘레'] <= 140) & (df_s['허리둘레'] >= 40)]

In [7]:
#diabete 식전혈당 (127이상:2, 100이상126미만:1, 정상 : 0)
def diabete (x) : 
    if (x >= 127) : return 2
    elif (x>=100 and x <= 126 ): return 1
    else : return 0
#bmi 단계  (0:정상, 1:과체중, 2:비만)
def bmi(h,w):
    bmi = w / ((h/100)**2)
    if (bmi >= 30) : return 2
    elif (bmi < 29 and bmi >= 25) : return 1
    else : return 0
    
#혈압 0:정상 , 1:전단계, 2:1기, 3:2기고혈압기고혈압  
def pulse(l,h):
    if (l >= 100 or h >= 160):
        return 3
    elif (90 <= l < 100) or (140 <= h < 160):
        return 2
    elif (80 <= l < 90) or (120 <= h < 140):
        return 1
    else:
        return 0
    
def bokbu(sex, waist) :
    if (sex == 1) and (waist >= 90) :
        return 1
    elif (sex == 2) and (waist >= 85) :
        return 2
    else : return 0
    

In [8]:
# 엄세진님 코드 참고했습니다.
df_s['혈당'] = df_s['식전혈당'].map(diabete)
df_s['고혈압'] = np.vectorize(pulse)(df_s['이완기혈압'], df_s['수축기혈압'])
df_s['bmi'] = np.vectorize(bmi)(df_s['신장'], df_s['체중'])
df_s['복부비만'] = np.vectorize(bokbu)(df_s['성별코드'], df_s['허리둘레'])
df_s.describe()

D:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2192: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


,성별코드,신장,체중,연령대코드,이완기혈압,수축기혈압,식전혈당,허리둘레,혈당,고혈압,bmi,복부비만
count,999666.000000,999666.000000,999666.000000,999666.000000,999652.000000,999651.000000,999625.000000,999666.000000,999666.000000,999666.000000,999666.000000,999666.000000
mean,1.469435,162.229165,63.270052,10.527908,76.051752,122.432110,100.457973,81.179611,0.449049,0.763584,0.339247,0.325536
std,0.499065,9.285150,12.520791,2.840880,9.894593,14.556586,24.251799,9.595587,0.629250,0.743347,0.563690,0.632687
min,1.000000,130.000000,25.000000,5.000000,30.000000,58.000000,25.000000,40.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,155.000000,55.000000,8.000000,70.000000,112.000000,88.000000,74.100000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,160.000000,60.000000,10.000000,76.000000,120.000000,96.000000,81.000000,0.000000,1.000000,0.000000,0.000000
75%,2.000000,170.000000,70.000000,13.000000,82.000000,131.000000,105.000000,87.800000,1.000000,1.000000,1.000000,0.000000
max,2.000000,190.000000,140.000000,18.000000,185.000000,273.000000,852.000000,140.000000,2.000000,3.000000,2.000000,2.000000


# 엔트로피

In [9]:
def Entropy(df, attribute) :
    list1  = list(df[attribute].unique())
    etp = 0
    for i in range(len(list1)):
        q1 = len(df[df[attribute] == list1[i]])/len(df[attribute])
        etp -=  q1*np.log2(q1)
    return etp

def Group_info(df,m_attribute,fix_attribute):
    list2 = list(df[m_attribute].unique())
    info_Entropy = Entropy(df,fix_attribute)
    info_data = 0
    for i in range (len(list2)) :
        q2 = df.loc[df[m_attribute] == list2[i]]
        info_data +=  (len(q2[m_attribute])/len(df[m_attribute]))* Entropy(q2,fix_attribute) 
    
    data = info_Entropy - info_data
    
    return data

info = np.array([("성별코드"),("고혈압"),("bmi"),("복부비만"),("연령대코드")])

for i in info:
    print(i, ":", Group_info(df_s, i, "혈당") * 1000)

성별코드 : 13.995870989997217
고혈압 : 30.179777895751325
bmi : 15.911702156371055
복부비만 : 21.854511238592387
연령대코드 : 47.7881958256976


In [10]:
Entropy(df_s,"혈당")

1.2234627560356184

### 지니 지수 https://lucy-the-marketer.kr/ko/growth/decision-tree-and-impurity/

In [11]:
def Gini(df, attribute) :
    list1  = list(df[attribute].unique())
    gini = 0
    for i in range(len(list1)):
        q1 = len(df[df[attribute] == list1[i]])/len(df[attribute])
        gini += (q1**2)
    return 1 - gini

def Group_Gini(df,m_attribute,fix_attribute):
    list2 = list(df[m_attribute].unique())
    info_data = 0
    for i in range (len(list2)) :
        q2 = df.loc[df[m_attribute] == list2[i]]
        info_data +=  (len(q2[m_attribute])/len(df[m_attribute]))* Gini(q2,fix_attribute) 
    
    return info_data

info = np.array([("성별코드"),("고혈압"),("bmi"),("복부비만"),("연령대코드")])

for i in info :
    print(i,":",Group_Gini(df_s, i,"혈당")*100)

성별코드 : 50.63366542723353
고혈압 : 49.904839405715045
bmi : 50.548242650914766
복부비만 : 50.348286979004484
연령대코드 : 49.41516144632993
